Method 1\
**Naïve Bayes classifier**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import numpy as np
import nltk
from sklearn.pipeline import Pipeline
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
# Load your movie dataset
df = pd.read_csv('./data/Training-dataset.csv')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
ul_list =[]
def get_wordnet_pos(nltk_tag):

    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
ul_list = []
for index, row in df.iterrows():
  words = word_tokenize(row['plot_synopsis'])
  Y=[]
  for word in words:
    if word.isalpha() and word.lower() not in stop_words:
      t=nltk.pos_tag([word])[0]
      wordnet_tagged = [t[0], get_wordnet_pos(t[1])]
      d=wordnet_tagged[1]
      if d !=None:
        Y.append(lemmatizer.lemmatize(word,d))
      else:
        Y.append(lemmatizer.lemmatize(word))
  ul_list.append(' '.join(Y))
df['processed_text']=ul_list


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Extract features and labels
X = df['processed_text']
y = df[['comedy', 'cult', 'flashback', 'historical', 'murder', 'revenge', 'romantic', 'scifi', 'violence']]
dftest = pd.read_csv('./data/Task-2-validation-dataset(1).csv')
ul_list =[]
def get_wordnet_pos(nltk_tag):

    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
ul_list = []
for index, row in dftest.iterrows():
  words = word_tokenize(row['plot_synopsis'])
  Y=[]
  for word in words:
    if word.isalpha() and word.lower() not in stop_words:
      t=nltk.pos_tag([word])[0]
      wordnet_tagged = [t[0], get_wordnet_pos(t[1])]
      d=wordnet_tagged[1]
      if d !=None:
        Y.append(lemmatizer.lemmatize(word,d))
      else:
        Y.append(lemmatizer.lemmatize(word))
  ul_list.append(' '.join(Y))
dftest['processed_text']=ul_list


In [4]:
import time
stime = time.time()
mlb1 = MultiLabelBinarizer()
ytr = mlb1.fit_transform(df[2:])

In [5]:
# Define the pipeline

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB

pipeline = make_pipeline(
    CountVectorizer(max_df=0.75, min_df=0.01, ngram_range=(1, 4)),
    OneVsRestClassifier(MultinomialNB(alpha=0.0001))
)

# Assuming df['processed_text'] contains your training data and labels
xtr = df['processed_text']
ytr = df.drop(['ID', 'title', 'plot_synopsis', 'processed_text'], axis=1)


In [6]:
# Training the pipeline
pipeline.fit(xtr, ytr)
etime=time.time()
print(etime-stime)

32.15846109390259


In [7]:
# Assuming dftest['processed_text'] contains your test data
X_val = dftest['processed_text']


In [8]:
# Assuming dftest['processed_text'] contains your test data
X_val = dftest['processed_text']


In [28]:
stime = time.time()
# Predicting probs
probs = pipeline.predict_proba(X_val)

# Apply threshold to determine labels
threshold = 0.84
predictedlabels = (probs >= threshold).astype(int)
etime=time.time()
print(etime-stime)

2.88242244720459


In [10]:
print(predictedlabels[0])

[0 0 0 0 0 0 1 0 0]


In [11]:
# Creating a DataFrame for predictions
predictedlabelsdf = pd.DataFrame(predictedlabels, columns=ytr.columns)
predictedlabelsdf['ID'] = dftest['ID'].values
predictedlabelsdf = predictedlabelsdf[['ID'] + [col for col in predictedlabelsdf.columns if col != 'ID']]

# Saving to CSV file
predictedlabelsdf.to_csv('./data/10949167-Task2-method-a-validation.csv', header=False, index=False)

In [12]:
#!python3 '/content/task2_eval_script_student_version(1).py' /content/10949167-Task2-method-a-validation.csv '/content/Task-2-validation-dataset(1).csv'

Class level: 
Class  1 precision: 0.2853 recall: 0.5086
Class  2 precision: 0.4194 recall: 0.5789
Class  3 precision: 0.4226 recall: 0.4456
Class  4 precision: 0.1200 recall: 0.3750
Class  5 precision: 0.7233 recall: 0.6885
Class  6 precision: 0.3419 recall: 0.5063
Class  7 precision: 0.5275 recall: 0.7276
Class  8 precision: 0.1205 recall: 0.3226
Class  9 precision: 0.5879 recall: 0.7167
----------------------------
Movie (document) level: 
Precision: 0.5001
Recall: 0.6281


In [29]:
# Extract features and labels
dftest1 = pd.read_csv('./data/Task-2-test-dataset2.csv')
ul_list =[]
index=0
row=0
for index, row in dftest1.iterrows():
  words = word_tokenize(row['plot_synopsis'])
  Y=[]
  for word in words:
    if word.isalpha() and word.lower() not in stop_words:
      t=nltk.pos_tag([word])[0]
      wordnet_tagged = [t[0], get_wordnet_pos(t[1])]
      d=wordnet_tagged[1]
      if d !=None:
        Y.append(lemmatizer.lemmatize(word,d))
      else:
        Y.append(lemmatizer.lemmatize(word))
  ul_list.append(' '.join(Y))
print(len(ul_list))
dftest1['processed_text']=ul_list
# Assuming dftest['processed_text'] contains your test data
X_test = dftest1['processed_text']
# # Predicting probs
import time
s_time = time.time()
probs = pipeline.predict_proba(X_test)
threshold = 0.84
predictedlabels = (probs >= threshold).astype(int)
e_time = time.time()
print(e_time-s_time)
# Creating a DataFrame for predictions
predictedlabelsdf = pd.DataFrame(predictedlabels, columns=ytr.columns)
predictedlabelsdf['ID'] = dftest1['ID'].values
predictedlabelsdf = predictedlabelsdf[['ID'] + [col for col in predictedlabelsdf.columns if col != 'ID']]

# Saving to CSV file
predictedlabelsdf.to_csv('./data/10949167-Task2-method-a.csv', header=False, index=False)

1200
1.855191946029663
